In [1]:
%cd ../..


/home/nazif/thesis/mirscribe


In [39]:
import pandas as pd
from scripts.utils import get_nucleotide_at_position, get_nucleotides_in_interval
from scripts.features import generate_is_mirna_column
import os

from scripts.globals import *


In [3]:
import pandas as pd

def load_vcf_into_df(file_path):
    """
    Load a VCF (Variant Call Format) file into a pandas DataFrame.
    
    Parameters:
    - file_path (str): The full path to the VCF file to be loaded.
    
    Returns:
    - DataFrame: A pandas DataFrame containing the VCF file data, with columns for 'chr', 'pos', 'id', 'ref', and 'alt'.
    
    Raises:
    - FileNotFoundError: If the specified file does not exist.
    - ValueError: If the file is not in the expected VCF format.
    - Exception: For other unforeseen errors during file loading.
    """
    # Ensure the file path ends with '.vcf', indicating it is a VCF file
    if not file_path.endswith('.vcf'):
        raise ValueError("The file specified does not appear to be a VCF file. Please provide a valid .vcf file.")

    try:
        return pd.read_csv(
            file_path,
            sep="\t",
            header=None,
            names=["chr", "pos", "id", "ref", "alt"],
            comment='#',
        )

    except FileNotFoundError as e:
        # Raise a more descriptive error if the file is not found
        raise FileNotFoundError(f"The file at {file_path} was not found.") from e
    
    except pd.errors.EmptyDataError as e:
        # Handle the case where the file is empty or does not contain expected data
        raise ValueError(
            f"The file at {file_path} is empty or not in the expected VCF format."
        ) from e
    except Exception as e:
        # Catch-all for other exceptions, with a generic error message
        print(f"An error occurred while trying to read the file at {file_path}: {e}")
        raise


In [43]:
file_path = "data/sample_vcfs/PD4248a.vcf"

filename = file_path.split("/")[-1].split(".")[0]


filename

'PD4248a'

In [24]:
df = load_vcf_into_df("data/sample_vcfs/PD4248a.vcf")

In [25]:
def augment_id(df):
    """Augment the 'id' column with additional information."""
    df['id'] += '_' + df['chr'] + '_' + df['pos'].astype(str) + '_' + df['ref'] + '_' + df['alt']


augment_id(df)
df.head()

,chr,pos,id,ref,alt
0,1,728633,PD4248a_1_728633_T_G,T,G
1,1,3717853,PD4248a_1_3717853_C_A,C,A
2,1,4609964,PD4248a_1_4609964_G_A,G,A
3,1,12695984,PD4248a_1_12695984_C_G,C,G
4,1,15821838,PD4248a_1_15821838_G_A,G,A


In [26]:
def compute_allele_lengths(df):
    """Compute lengths of reference and alternate alleles."""
    df["ref_len"] = df["ref"].apply(len)
    df["alt_len"] = df["alt"].apply(len)

compute_allele_lengths(df)
df.head()

,chr,pos,id,ref,alt,ref_len,alt_len
0,1,728633,PD4248a_1_728633_T_G,T,G,1,1
1,1,3717853,PD4248a_1_3717853_C_A,C,A,1,1
2,1,4609964,PD4248a_1_4609964_G_A,G,A,1,1
3,1,12695984,PD4248a_1_12695984_C_G,C,G,1,1
4,1,15821838,PD4248a_1_15821838_G_A,G,A,1,1


In [28]:
def fetch_nucleotides(df):
    # Function to fetch nucleotides based on the length of the affected sequences
    # if affected sequences are longer than 1, fetch nucleotides at the specified interval
    df.loc[df['ref_len'] > 1, 'fetched_nucleotides'] = df.apply(
        lambda x: get_nucleotides_in_interval(x['chr'], x['pos'], x["pos"]+x["ref_len"]-1), axis=1
    )
    # if affected sequences are equal to 1, fetch nucleotides at the specified pos
    df.loc[df['ref_len'] == 1, 'fetched_nucleotides'] = df.apply(
        lambda x: get_nucleotide_at_position(x['chr'], x['pos']), axis=1
    )
    # same but for 0 length
    df.loc[df['ref_len'] == 0, 'fetched_nucleotides'] = ""
    

fetch_nucleotides(df)
df.head()

,chr,pos,id,ref,alt,ref_len,alt_len,fetched_nucleotides
0,1,728633,PD4248a_1_728633_T_G,T,G,1,1,T
1,1,3717853,PD4248a_1_3717853_C_A,C,A,1,1,C
2,1,4609964,PD4248a_1_4609964_G_A,G,A,1,1,G
3,1,12695984,PD4248a_1_12695984_C_G,C,G,1,1,C
4,1,15821838,PD4248a_1_15821838_G_A,G,A,1,1,G


In [31]:
def compare_nucleotides(df):
    # Compare the fetched nucleotides with the reference nucleotides
    df["is_nucleotides_same"] = df["fetched_nucleotides"] == df["ref"]

compare_nucleotides(df)
df.head()

,chr,pos,id,ref,alt,ref_len,alt_len,fetched_nucleotides,is_nucleotides_same
0,1,728633,PD4248a_1_728633_T_G,T,G,1,1,T,True
1,1,3717853,PD4248a_1_3717853_C_A,C,A,1,1,C,True
2,1,4609964,PD4248a_1_4609964_G_A,G,A,1,1,G,True
3,1,12695984,PD4248a_1_12695984_C_G,C,G,1,1,C,True
4,1,15821838,PD4248a_1_15821838_G_A,G,A,1,1,G,True


In [32]:
def prepare_sequences(df):
    """
    Prepares reference indices, extended sequences, wild type, and mutated sequences for a given DataFrame.

    Parameters:
    - df (pd.DataFrame): The DataFrame containing the genetic information.
    - get_nucleotides_in_interval (callable): A function that takes chromosome ('chr'), start, and end positions as arguments and returns the nucleotide sequence.

    Returns:
    pd.DataFrame: The original DataFrame with added columns for reference start/end, upstream/downstream sequences, wild type sequence, and mutated sequence.
    """
    # Preparing reference indices
    df["ref_start"] = df["pos"]
    df["ref_end"] = df["pos"] + df["ref_len"]

    # Preparing extended sequences
    df['upstream_sequence'] = df.apply(lambda row: get_nucleotides_in_interval(row['chr'], row['ref_start'] - 30, row["ref_start"] - 1), axis=1)
    df['downstream_sequence'] = df.apply(lambda row: get_nucleotides_in_interval(row['chr'], row['ref_end'], row["ref_end"] + 29), axis=1)

    # Preparing wild type sequence
    df['sequence'] = df["upstream_sequence"] + df["ref"] + df["downstream_sequence"]

    # Preparing mutated sequence
    df['mutated_sequence'] = df["upstream_sequence"] + df["alt"] + df["downstream_sequence"]


prepare_sequences(df)


In [35]:
def import_pyensembl(grch):
    if grch not in [37, 38]:
        raise ValueError("grch must be either 37 or 38")

    ens_release = 75 if grch == 37 else 109
    
    import os

    from pyensembl import EnsemblRelease
    os.environ['PYENSEMBL_CACHE_DIR'] = PYENSEMBL_CACHE_DIR
    assembly = EnsemblRelease(ens_release)
    assembly.download()
    assembly.index()
    
    return assembly

grch37 = import_pyensembl(37)

INFO:pyensembl.sequence_data:Loaded sequence dictionary from /home/nazif/thesis/data/pyensembl/GRCh37/ensembl75/Homo_sapiens.GRCh37.75.cdna.all.fa.gz.pickle
INFO:pyensembl.sequence_data:Loaded sequence dictionary from /home/nazif/thesis/data/pyensembl/GRCh37/ensembl75/Homo_sapiens.GRCh37.75.ncrna.fa.gz.pickle
INFO:pyensembl.sequence_data:Loaded sequence dictionary from /home/nazif/thesis/data/pyensembl/GRCh37/ensembl75/Homo_sapiens.GRCh37.75.pep.all.fa.gz.pickle


In [38]:
df = generate_is_mirna_column(df, grch=37)

In [36]:
def generate_transcript_id_and_gene_name_columns(df, assembly):
    
    df['transcript_id'] = df.apply(lambda x: assembly.transcript_ids_at_locus(x['chr'], x['pos']), axis=1)
    # df["transcript_name"] = df.apply(lambda x: assembly.transcript_names_at_locus(x['chr'], x['pos']), axis=1)
    df["gene_name"] = df.apply(lambda x: assembly.gene_names_at_locus(x['chr'], x['pos']), axis=1)
    

generate_transcript_id_and_gene_name_columns(df, grch37)

In [44]:
def classify_and_save(df, file_name, save_path="results"):
    """Classifies mutations, saves to files, and logs the output."""

    case_1 = df[df.is_mirna == 0][["id", "sequence", "mutated_sequence"]]
    case_2 = df[df.is_mirna == 1][["id", "sequence", "mutated_sequence"]]

    # Create the directory if it does not exist
    os.makedirs(save_path, exist_ok=True)

    case_1.to_csv(f"{save_path}/{file_name}_case_1.csv", index=False)
    if not case_2.empty:
        print(f"{len(case_2)} case 2 mutations were found on {file_name}.")
        case_2.to_csv(f"{save_path}/{file_name}_case_2.csv", index=False)
    else:
        print(f"No case 2 mutations were found on {file_name}.")

In [45]:
classify_and_save(df, filename)

No case 2 mutations were found on PD4248a.
